In [ ]:
!pip install diffusers transformers torch torchvision accelerate safetensors huggingface_hub --quiet

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

prompt = "(raw photo, masterpiece, best quality, high detail), (8k, full frame, photorealistic, realistic, photo of a blue 1953 bel air."

image = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0]
image.save("output.png")

In [ ]:
# Pixar Cars

In [ ]:
!git clone https://github.com/replicate/cog-sdxl cog_sdxl

In [ ]:
import torch
from huggingface_hub import hf_hub_download
from diffusers import DiffusionPipeline
from cog_sdxl.dataset_and_utils import TokenEmbeddingsHandler
from diffusers.models import AutoencoderKL

pipe = DiffusionPipeline.from_pretrained(
        "stabilityai/stable-diffusion-xl-base-1.0",
        torch_dtype=torch.float16,
        variant="fp16",
).to("cuda")

pipe.load_lora_weights("fofr/sdxl-pixar-cars", weight_name="lora.safetensors")

text_encoders = [pipe.text_encoder, pipe.text_encoder_2]
tokenizers = [pipe.tokenizer, pipe.tokenizer_2]

embedding_path = hf_hub_download(repo_id="fofr/sdxl-pixar-cars", filename="embeddings.pti", repo_type="model")
embhandler = TokenEmbeddingsHandler(text_encoders, tokenizers)
embhandler.load_embeddings(embedding_path)
prompt="A photo of a blue <s0><s1> Pixar car with eyes, photography, 50mm"
images = pipe(
    prompt,
    cross_attention_kwargs={"scale": 0.8},
).images
#your output image
images[0]
image.save("output-pixar.png")


Free GPU Memory

In [21]:
torch.cuda.empty_cache()